# Work with JSON objects and files

- [Real Python tutorial](https://realpython.com/python-json/)
- [json standard library module](https://docs.python.org/3/library/json.html#)

In [269]:
import inspect
import json
import numpy as np
import pandas as pd

from pathlib import Path

from ecutils.historical_price_handling import sources_dict, ticker_dict, get_module_root_path

module_root = Path(get_module_root_path())

In [267]:
import ecutils

In [268]:
ecutils.__file__

'd:\\pyprojects\\ec-utils\\ecutils\\__init__.py'

In [276]:
inspect.getfile(get_module_root_path)

'd:\\pyprojects\\ec-utils\\ecutils\\historical_price_handling.py'

## Load the json file and Pretty Print its content

In [2]:
json_file = module_root / 'ecutils' / 'historical_prices_tickers.json'
assert json_file.is_file()

In [3]:
with open(json_file, 'r') as f:
    json_obj = json.load(f)

In [5]:
# print(json.dumps(json_obj, indent=4))

## Save the json file into a readable format (with indent)

In [43]:
# with open(json_file, mode='w') as fp:
#     json.dump(json_obj, fp, indent=4))

In [6]:
json_obj['S&P500']['tickers']

{'axitrader': 'US500', 'wsj': 'SPX', 'yahoo': '^GSPC'}

## Add parameters from Axitrader Product Specs into tickers json object

Load axitrader's specification and create a DataFrame `axitrader_specs` with relevant parameters. Index is the ticker symbol

In [228]:
axitrader_specs_directory = Path('D:\\PyProjects\\fx-bt\\data')
assert axitrader_specs_directory.is_dir()

specs = [f for f in axitrader_specs_directory.iterdir() if 'axitrader-product-spec' in f.name]
for f in specs:
    assert f.is_file()

specs

[WindowsPath('D:/PyProjects/fx-bt/data/axitrader-product-spec-commodity-cash-cfds.csv'),
 WindowsPath('D:/PyProjects/fx-bt/data/axitrader-product-spec-commodity-future-cfds.csv'),
 WindowsPath('D:/PyProjects/fx-bt/data/axitrader-product-spec-index-cash-cfds.csv'),
 WindowsPath('D:/PyProjects/fx-bt/data/axitrader-product-spec-index-future-cfds.csv'),
 WindowsPath('D:/PyProjects/fx-bt/data/axitrader-product-spec-margin-fx-contracts.csv')]

In [235]:
cols = ['Symbol', 'Market Description', 'Standard Margin Rate',
       'Account Leverage Factor', 'Currency', 'Contract Size per 1 Lot',
       '1 Lot Value per Pip', 'Indicative Spreads (Pips)',
       'Min mum Tick Increment', 'Min Trade Size (Lots)',
       'Max Trade Size (Lots)', '3 Day Roll', 'Type']
axitrader_specs = pd.DataFrame(columns=cols)
for spec in specs:
    df = pd.read_csv(spec)
    axitrader_specs = axitrader_specs.append(df.loc[:, cols], sort=False)

axitrader_specs.set_index(keys='Symbol', drop=True, inplace=True)
axitrader_specs_to_be_processed = True
display(axitrader_specs.sample(3))

,Market Description,Standard Margin Rate,Account Leverage Factor,Currency,Contract Size per 1 Lot,1 Lot Value per Pip,Indicative Spreads (Pips),Min mum Tick Increment,Min Trade Size (Lots),Max Trade Size (Lots),3 Day Roll,Type
Symbol,,,,,,,,,,,,
USDINR,US Dollar vs Indian Rupee 1 Month,3%,Yes,USD,100000,INR 1000 per 0.01,2.1,0.00010,0.01,20,Wednesday,Margin FX Contract
TRYJPY,Turkish Lira vs Japanese Yen,2%,Yes,TRY,100000,JPY 1000 per 0.01,1.3,0.00100,0.01,20,Wednesday,Margin FX Contract
EURUSD,Euro vs US Dollar,1%,Yes,EUR,100000,USD 10 per 0.0001,1.2,0.00001,0.01,100,Wednesday,Margin FX Contract


Make lists of tickers from json and from specs, a conversion dictionary, and view a few instruments to compare info in json and parameters in the spec

In [236]:
axitrader_specs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 142 entries, USOIL to ZARJPY
Data columns (total 12 columns):
Market Description           142 non-null object
Standard Margin Rate         142 non-null object
Account Leverage Factor      142 non-null object
Currency                     142 non-null object
Contract Size per 1 Lot      142 non-null object
1 Lot Value per Pip          142 non-null object
Indicative Spreads (Pips)    142 non-null float64
Min mum Tick Increment       142 non-null float64
Min Trade Size (Lots)        142 non-null float64
Max Trade Size (Lots)        142 non-null object
3 Day Roll                   142 non-null object
Type                         142 non-null object
dtypes: float64(3), object(9)
memory usage: 14.4+ KB


In [237]:
if axitrader_specs_to_be_processed:
    print('Processing dataframe:\n')
    axitrader_specs['Standard Margin Rate'] = axitrader_specs['Standard Margin Rate'].apply(lambda s: float(s[:-1])/100)
    axitrader_specs['Indicative Spreads (Pips)'] = axitrader_specs['Indicative Spreads (Pips)'].apply(lambda s: float(s))
    axitrader_specs['Min Trade Size (Lots)'] = axitrader_specs['Min Trade Size (Lots)'].apply(lambda s: float(s))
    axitrader_specs['Max Trade Size (Lots)'] = axitrader_specs['Max Trade Size (Lots)'].apply(lambda s: float(s))
    axitrader_specs_to_be_processed = False

axitrader_specs.info()

Processing dataframe:

<class 'pandas.core.frame.DataFrame'>
Index: 142 entries, USOIL to ZARJPY
Data columns (total 12 columns):
Market Description           142 non-null object
Standard Margin Rate         142 non-null float64
Account Leverage Factor      142 non-null object
Currency                     142 non-null object
Contract Size per 1 Lot      142 non-null object
1 Lot Value per Pip          142 non-null object
Indicative Spreads (Pips)    142 non-null float64
Min mum Tick Increment       142 non-null float64
Min Trade Size (Lots)        142 non-null float64
Max Trade Size (Lots)        142 non-null float64
3 Day Roll                   142 non-null object
Type                         142 non-null object
dtypes: float64(5), object(7)
memory usage: 14.4+ KB


In [238]:
security_json = json_obj.copy()

In [239]:
# View structure of one instrument in json
security_json["S&P500"]

{'name': 'S&P500 Cash CDF',
 'type': 'Index Cash CFDs',
 'description': 'US 500',
 'tickers': {'axitrader': 'US500', 'wsj': 'SPX', 'yahoo': '^GSPC'},
 'axitrader': {'Standard Margin Rate': 0.04,
  'Account Leverage Factor': 'Yes',
  'Currency': 'USD',
  'Contract Size per 1 Lot': 'USD 1 x Index',
  '1 Lot Value per Pip': 'USD 1 per 1.00',
  'Indicative Spreads (Pips)': 0.4,
  'Min mum Tick Increment': 0.01,
  'Min Trade Size (Lots)': 1.0,
  'Max Trade Size (Lots)': 5000.0,
  '3 Day Roll': 'Friday'}}

In [240]:
axitrader_ticker_to_instrument_key = dict([(instrument['tickers']['axitrader'], key) 
                                           for key, instrument in security_json.items()
                                           if instrument['tickers'].get('axitrader', None) is not None]
                                         )

list_axitrader_tickers_from_json = axitrader_ticker_to_instrument_key.keys()

list_axitrader_tickers_from_specs = axitrader_specs.index

tickers_in_both_lists = list(set(list_axitrader_tickers_from_json).intersection(set(list_axitrader_tickers_from_specs)))


print(f"Following tickers are in json but not in spec: {list(set(list_axitrader_tickers_from_json).difference(set(list_axitrader_tickers_from_specs)))}")
print(len(list_axitrader_tickers_from_json), len(list_axitrader_tickers_from_specs), len(tickers_in_both_lists))

Following tickers are in json but not in spec: ['USDHKD', 'BTCUSD']
60 142 58


In [241]:
for ticker in tickers_in_both_lists[0:1]:
    print('Original Info in json object:')
    display(security_json[axitrader_ticker_to_instrument_key[ticker]])
    print('Parameters to add:')
    display(axitrader_specs.loc[ticker,:])

Original Info in json object:


{'name': 'AUDCHF',
 'description': 'Australian Dollar vs Swiss Franc',
 'type': 'Margin FX Contract',
 'tickers': {'alphavantage': 'AUDCHF', 'axitrader': 'AUDCHF'},
 'axitrader': {'Standard Margin Rate': 0.01,
  'Account Leverage Factor': 'Yes',
  'Currency': 'AUD',
  'Contract Size per 1 Lot': 100000,
  '1 Lot Value per Pip': 'CHF 10 per 0.0001',
  'Indicative Spreads (Pips)': 1.9,
  'Min mum Tick Increment': 1e-05,
  'Min Trade Size (Lots)': 0.01,
  'Max Trade Size (Lots)': 50.0,
  '3 Day Roll': 'Wednesday'}}

Parameters to add:


Market Description           Australian Dollar vs Swiss Franc
Standard Margin Rate                                     0.01
Account Leverage Factor                                   Yes
Currency                                                  AUD
Contract Size per 1 Lot                                100000
1 Lot Value per Pip                         CHF 10 per 0.0001
Indicative Spreads (Pips)                                 1.9
Min mum Tick Increment                                  1e-05
Min Trade Size (Lots)                                    0.01
Max Trade Size (Lots)                                      50
3 Day Roll                                          Wednesday
Type                                       Margin FX Contract
Name: AUDCHF, dtype: object

In [242]:
axitrader_specs_params = ['Standard Margin Rate', 
                          'Account Leverage Factor',
                          'Currency',
                          'Contract Size per 1 Lot',
                          '1 Lot Value per Pip',
                          'Indicative Spreads (Pips)',
                          'Min mum Tick Increment',
                          'Min Trade Size (Lots)',
                          'Max Trade Size (Lots)',
                          '3 Day Roll']

In [243]:
# for ticker in tickers_in_both_lists[0:1]:
for ticker in tickers_in_both_lists:
    security_info = security_json[ticker_index2info_key[ticker]]
    security_info['description'] = axitrader_specs.loc[ticker, 'Market Description']
    security_info['type'] = axitrader_specs.loc[ticker, 'Type']
    axitrader_dict = {}
    for parameter in axitrader_specs_params:
        axitrader_dict[parameter] = axitrader_specs.loc[ticker, parameter]
    security_info['axitrader'] = axitrader_dict

In [244]:
security_json[ticker_index2info_key[tickers_in_both_lists[0]]]

{'name': 'AUDCHF',
 'description': 'Australian Dollar vs Swiss Franc',
 'type': 'Margin FX Contract',
 'tickers': {'alphavantage': 'AUDCHF', 'axitrader': 'AUDCHF'},
 'axitrader': {'Standard Margin Rate': 0.01,
  'Account Leverage Factor': 'Yes',
  'Currency': 'AUD',
  'Contract Size per 1 Lot': 100000,
  '1 Lot Value per Pip': 'CHF 10 per 0.0001',
  'Indicative Spreads (Pips)': 1.9,
  'Min mum Tick Increment': 1e-05,
  'Min Trade Size (Lots)': 0.01,
  'Max Trade Size (Lots)': 50.0,
  '3 Day Roll': 'Wednesday'}}

In [225]:
# print(json.dumps(security_json, indent=3))

## Safe new structure into `historical_prices_tickers.json`

In [245]:
json_file

WindowsPath('D:/PyProjects/ec-utils/ecutils/historical_prices_tickers.json')

In [263]:
save_file = False

In [264]:
if json_file.is_file():
    overwrite_file = False
    print(f"Are you sure you want to overwrite {json_file.name}? If yes, modify save_file in previous cell and then play next cell")

Are you sure you want to overwrite historical_prices_tickers.json? If yes, modify save_file in previous cell and then play next cell


In [266]:
if save_file:
    with open(json_file, mode='w') as fp:
        json.dump(json_obj, fp, indent=4)
else:
    print(f"{json_file.name} was not saved")

historical_prices_tickers.json was not saved


## New Section